# 05 – Feedback Loop: Tái tạo CoT dựa trên đánh giá

Vòng lặp tự động cải thiện prompt khi LogicalFlowMetric dưới ngưỡng.

In [ ]:
from deepeval import LLMTestCase, assert_test
from deepeval.metrics.geval import GEval
from langgraph.graph import StateGraph
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name='gpt-3.5-turbo')

logical_prompt = "Chấm 1 nếu chuỗi suy luận hợp lý; 0 nếu không:\n{actual_output}"
LogicalFlow = GEval(name="LogicalFlow", prompt=logical_prompt, threshold=0.7)

def build_agent(prompt: str):
    g = StateGraph(llm=llm)
    g.add_system_message(prompt)
    g.add_user_message("Tính căn bậc hai của 65?")
    return g

base_prompt = "Bạn là trợ lý toán học, giải thích từng bước."
for i in range(5):
    agent = build_agent(base_prompt)
    res = agent.run()
    cot = res.cot
    case = LLMTestCase(input="sqrt(65)", actual_output=cot)
    passed = assert_test(case, [LogicalFlow], raise_error=False)
    if passed:
        print(f"✅ Thành công ở vòng {i+1}")
        break
    reason = case.feedback[0].failure_reasons[0]
    print(f"❌ Vòng {i+1} thất bại: {reason}")
    base_prompt += f"\n# Chú ý sửa lỗi: {reason}" 